# Linear Regression Inputs

The following code defines three tensors (or nodes) to a linear regression model. Since they are of type `tf.placeholder` they need to be given a value in order to run a forward pass. From the TensorFlow [getting started](https://www.tensorflow.org/get_started/get_started) page

> A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.

In [1]:
import tensorflow as tf

x = tf.placeholder(dtype=tf.float32, name='x')
w = tf.placeholder(dtype=tf.float32, name='w')
b = tf.placeholder(dtype=tf.float32, name='b')

x, w, b

(<tf.Tensor 'x:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'w:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'b:0' shape=<unknown> dtype=float32>)

# Dump Graph

We create a TensorFlow [*session*](https://www.tensorflow.org/get_started/get_started) which encapsulates the control and state of the TensorFlow runtime followed by a `tf.summary.FileWriter` so we can visualize the computational graph in TensorBoard.

In [2]:
sess = tf.InteractiveSession()
writer = tf.summary.FileWriter('tensorboard/LR Inputs', sess.graph)
writer.flush()

sess, writer

(<tensorflow.python.client.session.InteractiveSession at 0x10acf5b70>,
 <tensorflow.python.summary.writer.writer.FileWriter at 0x10acf5b38>)

# Make Prediction

We combine our weight tensors `w` and `b` along with the input tensor `x` with tensorflow *operations* or *ops* to produce new tensors which represent the prediction of the linear regression model.

In [3]:
z = tf.multiply(x, w, name='z')
y_pred = tf.add(z, b, name='y_pred')

z, y_pred

(<tf.Tensor 'z:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'y_pred:0' shape=<unknown> dtype=float32>)

# Dump Graph

We create a new `tf.summary.FileWriter` with a different log destination so as to not overwrite the previous TensorBoard visualization.

In [4]:
writer = tf.summary.FileWriter('tensorboard/LR Forward', sess.graph)
writer.flush()

# Add Squared Loss

We define the last input `y` which is compared against $\hat{y}$ with a squared loss producing a final `loss` tensor.

In [5]:
y = tf.placeholder(dtype=tf.float32, name='y')

r = tf.subtract(y_pred, y, name='r')
loss = tf.square(r, name='loss')

y, r, loss

(<tf.Tensor 'y:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'r:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'loss:0' shape=<unknown> dtype=float32>)

# Dump Graph

In [6]:
writer = tf.summary.FileWriter('tensorboard/LR + SquaredLoss', sess.graph)
writer.flush()

# Get Gradients

TensorFlow provides machinery for computing derivatives. We can ask for the derivative of any tensor with respect to any other tensor in the computational graph, assuming the derivative can be computed.

In [7]:
dw = tf.gradients(loss, w, name='dw')[0]
db = tf.gradients(loss, b, name='db')[0]

dw, db

(<tf.Tensor 'dw/z_grad/Reshape_1:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'db/y_pred_grad/Reshape_1:0' shape=<unknown> dtype=float32>)

# Dump Graph

In [8]:
writer = tf.summary.FileWriter('tensorboard/LR + SquaredLoss + Gradients', sess.graph)
writer.flush()

# Forward Pass

Let's run a forward pass on our computational graph. To do this, we need to use the `sess.run()` function and pass it values for the input tensors (via the `feed_dict` argument) and which tensor(s) to return the value of (via the `fetches` argument).

In [9]:
x_ = y_ = w_ = b_ = 1

sess.run(fetches=[loss], feed_dict={x: x_, w: w_, b: b_, y: y_})

[1.0]

# Evaluating Part of our Computational Graph

Notice that whichever tensor we are interested in the value of, we only have to supply the values necessary to evaluate that tensor. For example, if we are interested in the value of `z`, we only need to supply the values of `x`, `w`, and `b` (not `y`!).

In [10]:
sess.run(fetches=[z], feed_dict={x: x_, w: w_, b: b_})

[1.0]

# Forward-Backward Pass

In [11]:
sess.run(fetches=[loss, dw, db], feed_dict={x: x_, w: w_, b: b_, y: y_})

[1.0, 2.0, 2.0]

# SGD Step

Once TensorFlow computes `dw` and `db` for us we can use them to step `w_` and `b_` to a location on the error surface with a smaller loss.

In [34]:
lr = 0.01

l, dw_, db_ = sess.run(fetches=[loss, dw, db], feed_dict={x: x_, w: w_, b: b_, y: y_})
w_ = w_ - lr*dw_
b_ = b_ - lr*db_

{'w': w_, 'b': b_, 'loss': l }

{'b': 0.69552773714065541, 'loss': 0.16593359, 'w': 0.69552773714065541}